# WEEK4_A_LDA_kor_news
* Author: Jongyoon Kim & Eu-Bin KIM
* 23rd of August 2021
* tlrndk123@gmail.com

## 목표!
1. LDA 모델을 학습시키는 데에 필요한 데이터 전처리 방법을 모색해봅시다. 문장에서 주제를 알게 해주는 형태소가 무엇인지, 문장에 어떤 부분들이 모델 학습에 불필요한지 생각해봅시다.
2. 여러 뉴스 기사들이 어떤 주제에 속하는 지 알아내봅시다. LDA 모델을 학습시켜 주제별 분류, 주제별 핵심 단어를 시각화해보는 것이 주된 목표입니다.



## Download packages

In [1]:
# pyLDAvis의 pandas dependency가 정확히 1.2.3이어야 시각화에서 에러가 발생하지 않습니다.
# 구글 코랩에 기본으로 설치된 버전은 1.1.5이기 때문에 상위 버전을 설치해줘야합니다.

# 최초실행시 runtime restart를 하라고 뜹니다. 
# 따라서 이 Cell만 실행후, runtime restart를 한뒤에 전체 Cells를 실행해주시기 바랍니다.

# runtime restart를 하지 않으면 반드시 에러가 발생합니다.

!pip3 install --upgrade pandas==1.2.3 

In [2]:
# 판다스 버전을 변경한 결과 아래 설치 진행시 에러, 워닝이 발생합니다.
# 대부분 구글 코랩, 텐서플로우 등 기본 설치된 패키지로 인해 발생하는 것이기 때문에
# 해당 튜토리얼과는 무관합니다. 그대로 진행하셔도 됩니다.
!pip3 -q install pyLDAvis konlpy

In [ ]:
import re
import urllib
import requests
import numpy as np
import pandas as pd

from datetime import datetime
from pprint import pprint

from konlpy.tag import Kkma 
kkma = Kkma()  # 형태소 분석기
from tqdm import tqdm

# Gensim
import gensim
import gensim.corpora as corpora  # 어휘구축을 위해서
from gensim.utils import simple_preprocess  # 토크나이즈
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()  # 노트북에서 시각화를 할 수 있도록

import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## Stop words prep. (불용어 준비)
Stop words are a set of commonly used words in a language. Examples of stop words in English are “a”, “the”, “is”, “are” and etc.

불용어는 자연어처리에 큰 의미가 없는 단어입니다. 그렇기에 이후에 데이터셋에서 제거하기 위해 미리 구축해둡니다.

[nltk.corpus.stopwords](https://www.nltk.org/book/ch02.html) -> 4.1 섹션을 참고하세요!

In [4]:
# 한국어 불용어 불러오기
f = requests.get('https://gist.githubusercontent.com/spikeekips/40eea22ef4a89f629abd87eed535ac6a/raw/4f7a635040442a995568270ac8156448f2d1f0cb/stopwords-ko.txt')
stop_words = f.content.decode("utf-8").split('\n')

In [5]:
stop_words[:20]

['가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '게다가',
 '게우다',
 '겨우']

## 한글 뉴스 데이터셋

실험에 사용할 데이터를 구축해봅시다. 한글 뉴스 데이터셋을 사용해볼겁니다. 

데이터는 dataframe(아래의 테이블같은 형태)로 준비합니다.

데이터가 csv, tsv 포맷이기 때문에 [pd.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)를 사용하면 됩니다. (tsv의 경우, sep파라미터를 '\t'로 꼭 설정하세요!)

target 칼럼이 의미하는 바는 다음과 같습니다.

정치(0), 경제(1), 사회(2), 생활/문화(3), 세계(4), 기술/IT(5), 연예(6), 스포츠(7)

In [6]:
# drop: 필요없는 column 제외.
df = pd.read_csv('https://gist.githubusercontent.com/ArtemisDicoTiar/7d91f779b2a9a7485009cb3f129fd711/raw/3a064717024895612f8684dbf8b7d67f5e70cae0/ko_news.csv')\
        .drop(columns=['Unnamed: 0', 'f_name'])\
        .rename(columns={'idx': 'target', 'txt': 'content'})

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   target   1600 non-null   int64 
 1   content  1600 non-null   object
dtypes: int64(1), object(1)
memory usage: 25.1+ KB


In [10]:
df.head(10)

,target,content
0,0,하필 지방선거 전날 북미회담… 야당엔 초대형 악재?\t한국당 “의도 개입된 날짜 선...
1,0,"[종합]김경수, 드루킹 '옥중편지'에 ""황당하고 어처구니 없는 소설""\t""거리낄 게..."
2,0,"홍준표 ""文정부, 얼마나 사정했으면 지선 하루전 북미회담…""\t지선 하루 앞 북미정..."
3,0,"文대통령 ""혁신성장, 국민 느끼기에 부족…중요한건 속도""\t""국제경쟁에서 경쟁국들 ..."
4,0,韓中 2년4개월만에 국방실무회의…中군용기 KADIZ침범 방지 요구(종합)\t국방부 ...
5,0,"판 깨지 않으면서… 김정은, 아슬아슬한 ‘줄타기 외교’\t패싱 우려한 中 심리 이용..."
6,0,"19일만에 마주앉는 남북, '판문점 선언' 이행 로드맵 짠다\t고위급회담 16일 개..."
7,0,"[종합]文대통령 ""해외소득·역외탈세에 국민 분노···반드시 근절""\t""해외재산 도피..."
8,0,"남경필 이틀째 강공모드…이재명 ""엄정 법적대응""\t두 후보, '욕설 음성파일' 놓고..."
9,0,반격한 김계관\t[한겨레] 6자회담 대표 지낸 외교 전문가\n\n과거 볼턴과 상대하...


## Data cleaning - email remove, newline character remove

Clean data is important in every cases especially in this Machine/Deep Learning field.

The newsgroup data includes emails which is unnecessary for topic modeling.

Also, "\n" means newline character that changes line which is also meaningless for the topic modeling.

위에서 불러온 데이터를 조금 정리해봅시다.

데이터 분석, 모델링에 불필요한 이메일주소, newline 문자(\n)을 제거합시다.

정규표현식 re 모듈을 이용해 지워 볼겁니다. [정규표현식 입문](https://wikidocs.net/1669)

In [13]:
# Convert to list
# 데이터프레임의 함수를 사용해서 전처리를 하기에는 복잡하다
# 그래서 리스트로 바꿔서 전처리를 하겠습니다
data = df.content.values.tolist()

In [16]:
for element in data[:2]:
  # 빈공간 - \n\n\n (세줄) -> new line character 제거
  # 기사 - 기자들이 마지막에 이메일을 남기는 부분이 있을수도 있다 -> 이메일 제거
  # 따옴표도 제거 
  # 불용어를 제거
  print(element)
  print("=" * 200)

하필 지방선거 전날 북미회담… 야당엔 초대형 악재?	한국당 “의도 개입된 날짜 선정” 민주당 “평화 여는 역사적 계기”



6·13 지방선거 전날인 다음 달 12일에 북·미 정상회담 개최가 확정되자 여야의 표정은 크게 엇갈렸다. 더불어민주당은 북·미 정상회담이 지방선거에서 호재로 작용할 것이라는 기대감을 감추지 않았다. 자유한국당과 바른미래당은 국제 이슈가 지방선거에 큰 영향을 주지 않을 것이라고 평가절하했다. 특히 한국당은 겉으로는 “미·북 정상회담의 성공을 기대한다”고 밝혔으나 당내에서는 “왜 하필 그날 열리는 것이냐”는 볼멘소리가 흘러나왔다.

추미애 민주당 대표는 11일 국회에서 열린 최고위원회의에서 “북·미 정상회담은 한·미 정상회담에서 열리기 시작한 평화의 문이 활짝 열리는 역사적 계기가 될 것”이라고 의미를 부여했다.

민주당은 북·미 정상회담에서 가시적 성과가 나온다면 민주당의 상승세가 더욱 거세질 것이라고 전망했다. 민주당 핵심 관계자는 “한반도 평화 모드에 대한 기대감이 이미 당 지지율에 반영됐다고 봐야 한다”면서도 “회담 성과가 예상을 뛰어넘을 경우 지방선거에서 엄청난 파괴력을 발휘할 것”이라고 말했다. 하지만 회담의 결과가 기대에 못 미칠 경우 역풍이 불 수 있다는 걱정도 없지 않다.

한국당은 환영 의사를 나타내면서도 의심의 시선을 감추지는 않았다. 장제원 수석대변인은 “지방선거 직전에 미·북 정상회담이 확정된 것에 대해 문재인정부의 정치적 의도가 개입된 것은 아닌지 의심스러운 면도 없지는 않다”면서도 “미·북 정상회담에서 핵 폐기가 합의된다면 환영할 것”이라고 밝혔다.

홍준표 대표는 경북 경주와 대구에서 잇따라 열린 경북도당·대구시당 필승결의대회에 참석해 “저 사람들(여권)은 지방선거를 어떻게 하면 남북 평화 쇼로 치를까 하는 오로지 그 생각밖에 없다”고 비판했다. 이어 “다음 주 미국 백악관에 북·미 회담에서 북한에 어떤 요구를 해야 할지 하는 한국당의 입장을 담은 공개서한을 직접 보낼 것”이라고 강조했다. 야권에서는 북·미 정상

### 이메일 주소 제거

이메일은 {영어 문자열과 숫자의 조합}@{영어 문자열과 숫자} 의 형태를 띄고 있습니다.

<br/>

#### 정규표현식에서 
whitespace(아무런 문자열이 적혀 있지 않은 공간)를 제외한 모든 문자와 매칭되는 문자 클래스는 "\S"입니다.

해당 문자 혹은 문자 클래스의 반복은 바로 뒤에 특수 메타문자를 이용해 표현됩니다.

0번 혹은 그 이상 반복을 의미하는 "*", 1번 혹은 그 이상 반복을 의미하는 "+"가 있습니다.

이메일의 형태이나 빈 주소 (예: @.com) 역시도 제거 대상이기 때문에 0번 그 이상 반복인 "*"을 사용해 \S의 반복을 표현해줍시다.

whitespace에 해당되는 문자클래스는 "\s"입니다.

이메일 주소 뒤에 빈공간 (탭, blankspace: 스페이스바 한번, \n: 줄바꿈 표시)가 올수 있기 때문에 이 역시 같이 제거해줍시다.

반복 횟수는 해당 whitespace하나가 올수도 안올 수도 있기 때문에 0번 혹은 1번 반복을 의미하는 "?" 메타 문자를 뒤에 넣어줍니다.
- ?가 있는 경우 - "non-greedy" 매칭.
- 종윤님 확인 부탁!

이제 작성한 정규표현식(\S*@\S*\s?)에 매칭된 모든 문자를 지워야겠죠?

[re.sub](https://docs.python.org/3/library/re.html#re.sub)을 이용해 정규표현식에 매칭되는 문자를 정규표현식 뒤에 오는 문자로 바꿔줍니다.
re.sub(정규표현식, 변경 목표값, 변경 대상 문자열)

In [18]:
# <greedy matching vs. non-greedy matching>
sent = "제 이메일 주소는 tlrndk123@gmail.com 1111111."
pattern_1 = re.compile('\S*@\S*\s[0-9]*?')  # non-greedy ~= lazy. 
pattern_2 = re.compile('\S*@\S*\s[0-9]*')
print(pattern_1.search(sent))
print(pattern_2.search(sent))

<re.Match object; span=(10, 30), match='tlrndk123@gmail.com '>
<re.Match object; span=(10, 37), match='tlrndk123@gmail.com 1111111'>


In [22]:
# Remove Emails
# (띄어쓰기(whitespace)가 아닌 모든 글자가 0개 이상)@(띄어쓰기(whitespace)가 아닌 모든 글자가 0개 이상)
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

### newline character, 작은 따옴표 제거

이메일 제거와 마찬가지로 새로운 줄 생성을 하나의 띄어쓰기로 바꿔줍시다.

문장 줄 바꿈은 사람이 글을 읽는 데 편하기 위해 써 놓은 것이기 때문이죠.

빈 공간, whitespace에 해당되는 문자 클래스 "\s"와 1회 이상 반복을 의미하는 메타 문자 "+"를 이용해 정규표현식을 작성합니다.

마찬가지로 re.sub 함수를 이용해 ' ' 빈공간 하나로 변환합니다.

작은 따옴표 역시 제거 해줍시다. 작은 따옴표는 '를 적어줍니다. 다만 컴퓨터는 코드를 이해할때 작은 따옴표를 문자 하나 (예: 'A')가 시작하는 기호로 이해하기 때문에 앞에 escape문자인 역슬래시를 넣어줍니다. 이 경우는 빈공간 없이 그대로 제거합니다.

In [21]:
# Remove new line characters
# \n - 줄바꿈 캐릭터.
data = [re.sub('\s+', ' ', sent) for sent in data]

In [26]:
print(data[0])

하필 지방선거 전날 북미회담… 야당엔 초대형 악재? 한국당 “의도 개입된 날짜 선정” 민주당 “평화 여는 역사적 계기” 6·13 지방선거 전날인 다음 달 12일에 북·미 정상회담 개최가 확정되자 여야의 표정은 크게 엇갈렸다. 더불어민주당은 북·미 정상회담이 지방선거에서 호재로 작용할 것이라는 기대감을 감추지 않았다. 자유한국당과 바른미래당은 국제 이슈가 지방선거에 큰 영향을 주지 않을 것이라고 평가절하했다. 특히 한국당은 겉으로는 “미·북 정상회담의 성공을 기대한다”고 밝혔으나 당내에서는 “왜 하필 그날 열리는 것이냐”는 볼멘소리가 흘러나왔다. 추미애 민주당 대표는 11일 국회에서 열린 최고위원회의에서 “북·미 정상회담은 한·미 정상회담에서 열리기 시작한 평화의 문이 활짝 열리는 역사적 계기가 될 것”이라고 의미를 부여했다. 민주당은 북·미 정상회담에서 가시적 성과가 나온다면 민주당의 상승세가 더욱 거세질 것이라고 전망했다. 민주당 핵심 관계자는 “한반도 평화 모드에 대한 기대감이 이미 당 지지율에 반영됐다고 봐야 한다”면서도 “회담 성과가 예상을 뛰어넘을 경우 지방선거에서 엄청난 파괴력을 발휘할 것”이라고 말했다. 하지만 회담의 결과가 기대에 못 미칠 경우 역풍이 불 수 있다는 걱정도 없지 않다. 한국당은 환영 의사를 나타내면서도 의심의 시선을 감추지는 않았다. 장제원 수석대변인은 “지방선거 직전에 미·북 정상회담이 확정된 것에 대해 문재인정부의 정치적 의도가 개입된 것은 아닌지 의심스러운 면도 없지는 않다”면서도 “미·북 정상회담에서 핵 폐기가 합의된다면 환영할 것”이라고 밝혔다. 홍준표 대표는 경북 경주와 대구에서 잇따라 열린 경북도당·대구시당 필승결의대회에 참석해 “저 사람들(여권)은 지방선거를 어떻게 하면 남북 평화 쇼로 치를까 하는 오로지 그 생각밖에 없다”고 비판했다. 이어 “다음 주 미국 백악관에 북·미 회담에서 북한에 어떤 요구를 해야 할지 하는 한국당의 입장을 담은 공개서한을 직접 보낼 것”이라고 강조했다. 야권에서는 북·미 정상회담에 대한 

In [28]:
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [32]:
# 특수 따옴표 제거
data = [re.sub("“", "", sent) for sent in data]  # 시작 따옴표  
data = [re.sub("”", "", sent) for sent in data]  # 마침 따옴표

pprint(data[:1])

# 이정무14:54
# 미*북, 북*미 가ㅏ 같다는건 어떻게 아는건가요?
# 속에 있는 미, 북이 다른 문서에서 나타나는 미, 북과 동일하다는 것을 구두점을 제거하면 알려줄수 있다. 
# 미*북, 북*미: 미=0, 북=0
# 미,북, 북,미 미=2, 북=2.
# 북, 미 정상회담

['하필 지방선거 전날 북미회담… 야당엔 초대형 악재? 한국당 의도 개입된 날짜 선정 민주당 평화 여는 역사적 계기 6·13 지방선거 전날인 '
 '다음 달 12일에 북·미 정상회담 개최가 확정되자 여야의 표정은 크게 엇갈렸다. 더불어민주당은 북·미 정상회담이 지방선거에서 호재로 '
 '작용할 것이라는 기대감을 감추지 않았다. 자유한국당과 바른미래당은 국제 이슈가 지방선거에 큰 영향을 주지 않을 것이라고 평가절하했다. '
 '특히 한국당은 겉으로는 미·북 정상회담의 성공을 기대한다고 밝혔으나 당내에서는 왜 하필 그날 열리는 것이냐는 볼멘소리가 흘러나왔다. '
 '추미애 민주당 대표는 11일 국회에서 열린 최고위원회의에서 북·미 정상회담은 한·미 정상회담에서 열리기 시작한 평화의 문이 활짝 열리는 '
 '역사적 계기가 될 것이라고 의미를 부여했다. 민주당은 북·미 정상회담에서 가시적 성과가 나온다면 민주당의 상승세가 더욱 거세질 것이라고 '
 '전망했다. 민주당 핵심 관계자는 한반도 평화 모드에 대한 기대감이 이미 당 지지율에 반영됐다고 봐야 한다면서도 회담 성과가 예상을 '
 '뛰어넘을 경우 지방선거에서 엄청난 파괴력을 발휘할 것이라고 말했다. 하지만 회담의 결과가 기대에 못 미칠 경우 역풍이 불 수 있다는 '
 '걱정도 없지 않다. 한국당은 환영 의사를 나타내면서도 의심의 시선을 감추지는 않았다. 장제원 수석대변인은 지방선거 직전에 미·북 '
 '정상회담이 확정된 것에 대해 문재인정부의 정치적 의도가 개입된 것은 아닌지 의심스러운 면도 없지는 않다면서도 미·북 정상회담에서 핵 '
 '폐기가 합의된다면 환영할 것이라고 밝혔다. 홍준표 대표는 경북 경주와 대구에서 잇따라 열린 경북도당·대구시당 필승결의대회에 참석해 저 '
 '사람들(여권)은 지방선거를 어떻게 하면 남북 평화 쇼로 치를까 하는 오로지 그 생각밖에 없다고 비판했다. 이어 다음 주 미국 백악관에 '
 '북·미 회담에서 북한에 어떤 요구를 해야 할지 하는 한국당의 입장을 담은 공개서한을 직접 보낼 것이

## Tokenize words + clean-up text (텍스트 정리 + list로 변환)
Now the ecah sentence is cleaned-up by removing punctuations and unnecessary characters. Then the sentence is expressed as a list.

문장을 들여다보면 언어 그자체의 의미는 없는 구두점들이 있습니다. 가령 (괄호)나 온점. 반점, 이 이에 해당합니다.

이 부분을 직접 정규표현식(regex)를 사용해 구현할 수도 있지만 우리는 바쁜 사람들이기 때문에 gensim에서 만들어준 [simple_preprocess](https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess)라는 유틸을 사용해서 제거합시다! :)

In [34]:
def sent_to_words(sentences):
    for sentence in sentences:
        # yield = Generator.
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  
        # deacc=True removes punctuations
        # gensim's simple_preprocess does the job that we desire.

# 토크나이즈를 함과 동시에, 구두점을 제거.
# 종윤님 확인 부탁!
data_words = list(sent_to_words(data))

# data_words = List[str] -> List[List[str]]
print(data_words[:1])

[['하필', '지방선거', '전날', '북미회담', '야당엔', '초대형', '악재', '한국당', '의도', '개입된', '날짜', '선정', '민주당', '평화', '여는', '역사적', '계기', '지방선거', '전날인', '다음', '일에', '정상회담', '개최가', '확정되자', '여야의', '표정은', '크게', '엇갈렸다', '더불어민주당은', '정상회담이', '지방선거에서', '호재로', '작용할', '것이라는', '기대감을', '감추지', '않았다', '자유한국당과', '바른미래당은', '국제', '이슈가', '지방선거에', '영향을', '주지', '않을', '것이라고', '평가절하했다', '특히', '한국당은', '겉으로는', '정상회담의', '성공을', '기대한다고', '밝혔으나', '당내에서는', '하필', '그날', '열리는', '것이냐는', '볼멘소리가', '흘러나왔다', '추미애', '민주당', '대표는', '국회에서', '열린', '최고위원회의에서', '정상회담은', '정상회담에서', '열리기', '시작한', '평화의', '문이', '활짝', '열리는', '역사적', '계기가', '것이라고', '의미를', '부여했다', '민주당은', '정상회담에서', '가시적', '성과가', '나온다면', '민주당의', '상승세가', '더욱', '거세질', '것이라고', '전망했다', '민주당', '핵심', '관계자는', '한반도', '평화', '모드에', '대한', '기대감이', '이미', '지지율에', '반영됐다고', '봐야', '한다면서도', '회담', '성과가', '예상을', '뛰어넘을', '경우', '지방선거에서', '엄청난', '파괴력을', '발휘할', '것이라고', '말했다', '하지만', '회담의', '결과가', '기대에', '미칠', '경우', '역풍이', '있다는', '걱정도', '없지', '않다', '한국당은', '환영', '의사를', '나타내면서도', '의심의', '시선을', '감추지는', '않았다', '장제원', '수석대

In [35]:
# 데이터를 한번에 로드하는 것이 아니라, 나중에 로드를 하고 싶을 때, generator를 사용한다.
# lazy loading. 
gen = sent_to_words(data)

In [ ]:
print(gen)
for elem in gen:
  print(elem)

## Creating Bi-gram, Tri-gram models

Bi-gram: two words frequently occuring together in the document. 두개 단어로 구성되는 collocaiton(연어)
Tri-gram: same as bigram but three words.

가장 자주 쓰이는 단어 두개, 세개 혹은 그 이상을 묶어주는 모델을 만들어봅시다. 다행히도 gensim이 해당 모델을 제공해줍니다. [gensim의 Pharases](https://radimrehurek.com/gensim/models/phrases.html) 모델을 사용합시다. 

여기서 말하는 단어쌍은 다음과 같은 겁니다.
* happy + birthday -> happy_birthday
* oil + leak -> oil_leak
* maryland + college + park -> maryland_college_park

아래 결과를 보면 nntp_posting_host로 세개의 단어가 뭉쳐 있는 것을 볼 수 있습니다.

이렇게 단어를 뭉쳐서 collocation형태로 표현하는 이유는 다음과 같은 현상 때문입니다.

Uni-grams
* topic1 -scuba,water,vapor,diving
* topic2 -dioxide,plants,green,carbon

Bi-gram topics
* topic1 -scuba diving,water vapor
* topic2 -green plants,carbon dioxide

In [37]:
# Build the bigram and trigram models
# gensim's Phrases model can implement the bigram, trigram and more.
# higher threshold fewer phrases.
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['하필', '지방선거', '전날', '북미회담', '야당엔', '초대형', '악재', '한국당', '의도', '개입된', '날짜', '선정', '민주당', '평화', '여는', '역사적', '계기', '지방선거', '전날인', '다음', '일에', '정상회담', '개최가', '확정되자', '여야의', '표정은', '크게', '엇갈렸다', '더불어민주당은', '정상회담이', '지방선거에서', '호재로', '작용할', '것이라는', '기대감을', '감추지', '않았다', '자유한국당과', '바른미래당은', '국제', '이슈가', '지방선거에', '영향을', '주지', '않을', '것이라고', '평가절하했다', '특히', '한국당은', '겉으로는', '정상회담의', '성공을', '기대한다고', '밝혔으나', '당내에서는', '하필', '그날', '열리는', '것이냐는', '볼멘소리가', '흘러나왔다', '추미애', '민주당', '대표는', '국회에서_열린', '최고위원회의에서', '정상회담은', '정상회담에서', '열리기', '시작한', '평화의', '문이', '활짝', '열리는', '역사적', '계기가', '것이라고', '의미를', '부여했다', '민주당은', '정상회담에서', '가시적', '성과가', '나온다면', '민주당의', '상승세가', '더욱', '거세질', '것이라고_전망했다', '민주당', '핵심', '관계자는', '한반도_평화', '모드에', '대한', '기대감이', '이미', '지지율에', '반영됐다고', '봐야', '한다면서도', '회담', '성과가', '예상을', '뛰어넘을', '경우', '지방선거에서', '엄청난', '파괴력을', '발휘할', '것이라고', '말했다', '하지만', '회담의', '결과가', '기대에', '미칠', '경우', '역풍이', '있다는', '걱정도', '없지', '않다', '한국당은', '환영', '의사를', '나타내면서도', '의심의', '시선을', '감추지는', '않았다', '장제원', '수석대변인은', '지방선

In [38]:
print(data_words[0])

['하필', '지방선거', '전날', '북미회담', '야당엔', '초대형', '악재', '한국당', '의도', '개입된', '날짜', '선정', '민주당', '평화', '여는', '역사적', '계기', '지방선거', '전날인', '다음', '일에', '정상회담', '개최가', '확정되자', '여야의', '표정은', '크게', '엇갈렸다', '더불어민주당은', '정상회담이', '지방선거에서', '호재로', '작용할', '것이라는', '기대감을', '감추지', '않았다', '자유한국당과', '바른미래당은', '국제', '이슈가', '지방선거에', '영향을', '주지', '않을', '것이라고', '평가절하했다', '특히', '한국당은', '겉으로는', '정상회담의', '성공을', '기대한다고', '밝혔으나', '당내에서는', '하필', '그날', '열리는', '것이냐는', '볼멘소리가', '흘러나왔다', '추미애', '민주당', '대표는', '국회에서', '열린', '최고위원회의에서', '정상회담은', '정상회담에서', '열리기', '시작한', '평화의', '문이', '활짝', '열리는', '역사적', '계기가', '것이라고', '의미를', '부여했다', '민주당은', '정상회담에서', '가시적', '성과가', '나온다면', '민주당의', '상승세가', '더욱', '거세질', '것이라고', '전망했다', '민주당', '핵심', '관계자는', '한반도', '평화', '모드에', '대한', '기대감이', '이미', '지지율에', '반영됐다고', '봐야', '한다면서도', '회담', '성과가', '예상을', '뛰어넘을', '경우', '지방선거에서', '엄청난', '파괴력을', '발휘할', '것이라고', '말했다', '하지만', '회담의', '결과가', '기대에', '미칠', '경우', '역풍이', '있다는', '걱정도', '없지', '않다', '한국당은', '환영', '의사를', '나타내면서도', '의심의', '시선을', '감추지는', '않았다', '장제원', '수석대변

## Remove stopwords, Make Bigrams & POS-tagging

* 이제 불용어를 제거하고
* 각 문서별로 자주 쓰이는 두개 단어를 추출하고
* 텍스트 분류기를 만들어봅시다.
  * 텍스트 분류기는 텍스트 내 특정 품사만을 걸러줍니다.
  * 이 튜토리얼에서는 NOUN(명사), ADJ(형용사), VERB(동사), ADV(부사)만 걸러집니다.

In [39]:
target_tags = [
    'NNG',  # 일반 명사
    'NNP',  # 고유 명사
    'NNB',  # 의존 명사
    'NR',  # 수사
    'NP',  # 대명사
    'VV',  # 동사
    'VA',  # 형용사
    'MAG',  # 일반 부사
    'MAJ',  # 접속 부사
]

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# 두 단어로 구성되는 연어를 찾아 토큰을 재구성
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

# 세 단어로 구성되는 연어를 찾아 토큰을 재구성
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# 표제어 추출은 아님.
# # 원형으로 정규화. 
# def lemmatization(texts, allowed_postags=target_tags):
#     results = list()
#     for text in texts:
#         # 품사추출만 진행.
#         results.append([s for s, t in kkma.pos(' '.join(text)) if t in allowed_postags and len(s) > 1])
#     return results

def filter_pos(texts, allowed_postags=target_tags):
    results = list()
    for text in tqdm(texts):
        # 품사추출만 진행.
        results.append([s for s, t in kkma.pos(' '.join(text)) if t in allowed_postags and len(s) > 1])
    return results

In [40]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# # Do lemmatization keeping only noun, adj, vb, adv
# # lemmatization 작업은 약 10분정도 소요됩니다! 아래 상태창을 보고 10분정도까지는 기다려주세요.
# data_lemmatized = lemmatization(data_words_bigrams)
## data_filtered = filter_pos(data_words_bigrams)
data_lemmatized = data_words_bigrams
print(data_lemmatized[:1])

[['하필', '지방선거', '전날', '북미회담', '야당엔', '초대형', '악재', '한국당', '의도', '개입된', '날짜', '선정', '민주당', '평화', '여는', '역사적', '계기', '지방선거', '전날인', '일에', '정상회담', '개최가', '확정되자', '여야의', '표정은', '크게', '엇갈렸다', '더불어민주당은', '정상회담이', '지방선거에서', '호재로', '작용할', '것이라는', '기대감을', '감추지', '않았다', '자유한국당과', '바른미래당은', '국제', '이슈가', '지방선거에', '영향을', '주지', '않을', '것이라고', '평가절하했다', '특히', '한국당은', '겉으로는', '정상회담의', '성공을', '기대한다고', '밝혔으나', '당내에서는', '하필', '그날', '열리는', '것이냐는', '볼멘소리가', '흘러나왔다', '추미애', '민주당', '대표는', '국회에서_열린', '최고위원회의에서', '정상회담은', '정상회담에서', '열리기', '시작한', '평화의', '문이', '활짝', '열리는', '역사적', '계기가', '것이라고', '의미를', '부여했다', '민주당은', '정상회담에서', '가시적', '성과가', '나온다면', '민주당의', '상승세가', '더욱', '거세질', '것이라고_전망했다', '민주당', '핵심', '관계자는', '한반도_평화', '모드에', '대한', '기대감이', '이미', '지지율에', '반영됐다고', '봐야', '한다면서도', '회담', '성과가', '예상을', '뛰어넘을', '경우', '지방선거에서', '엄청난', '파괴력을', '발휘할', '것이라고', '말했다', '회담의', '결과가', '기대에', '미칠', '경우', '역풍이', '있다는', '걱정도', '없지', '않다', '한국당은', '환영', '의사를', '나타내면서도', '의심의', '시선을', '감추지는', '않았다', '장제원', '수석대변인은', '지방선거', '직전에', '

## Create dictionary and corpus for topic modeling

이제 준비된 데이터를 컴퓨터가 이해하기 쉬운 숫자로 바꿔줍시다. 이후에 사람이 직접 디버깅하며 무슨 내용인지 이해 할 수 있게 사전([gensim.corpora.Dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html))도 준비해두고요.

Gensim uses unique id for each word in the document.


In [41]:
# Create Dictionary
# 각 단어에 대응하는 정수가 무엇인지 찾아준다
#
id2word = corpora.Dictionary(data_lemmatized)
# id2word[3] -> 하다 
# id2word[1] -> 자동차


In [42]:
print(id2word[0])
print(id2word[1])
print(id2word[2])
print(id2word.token2id["가뜩이나"])
print(id2word[19])

가뜩이나
가시적
감추지
0
경우


In [43]:
sent = ["안녕", "안녕", "하세요"]
bow = id2word.doc2bow(sent)  # bag-of-words 표현. 
print(bow)

[(29299, 2), (37681, 1)]


In [44]:
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
# List[str] -> Bow.
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])
# (word's unique id, word_frequnecy)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 5), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 3), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 2), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 3), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 2), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 2), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

In [45]:
# 0번째 단어는 '가뜩이나'입니다.
id2word[corpus[0][0][0]]

'가뜩이나'

In [46]:
[[(id, id2word[id], freq) for id, freq in cp][:10] for cp in corpus[:1]]
# (word's unique id, human readable word, frequnecy)
# 상위 10개 단어만 출력합니다.

[[(0, '가뜩이나', 1),
  (1, '가시적', 1),
  (2, '감추지', 1),
  (3, '감추지는', 1),
  (4, '강조했다', 1),
  (5, '개입된', 2),
  (6, '개최가', 1),
  (7, '거세질', 1),
  (8, '걱정도', 1),
  (9, '것에_대해', 1)]]

# Build Topic Model

토픽 모델을 준비합시다. gensim에서 제공해주는 ldamodel을 가져다 씁시다 XD

[gensim.models.ldamodel](https://radimrehurek.com/gensim/models/ldamodel.html)


~~~python
class gensim.models.ldamodel.LdaModel(corpus=None, 
                                     num_topics=100, 
                                     id2word=None, 
                                     distributed=False, 
                                     chunksize=2000, 
                                     passes=1, 
                                     update_every=1, 
                                     alpha='symmetric', 
                                     eta=None, 
                                     decay=0.5, 
                                     offset=1.0, 
                                     eval_every=10, 
                                     iterations=50, 
                                     gamma_threshold=0.001, 
                                     minimum_probability=0.01, 
                                     random_state=None, 
                                     ns_conf=None, 
                                     minimum_phi_value=0.01, 
                                     per_word_topics=False, 
                                     callbacks=None, 
                                     dtype=<class 'numpy.float32'>)
~~~

In [47]:
lda_model = gensim.models.ldamodel.LdaModel(iterations=200,
                                            corpus=corpus,  # bag-of-words의 리스트.
                                            id2word=id2word,  # 정수 인코딩된 단어를 디코딩하기 위해선, dictionary객체가 필요.
                                            num_topics=8,  #만약에 토픽이 8개라면, 그러면 그 토픽은 무엇이니?
                                            random_state=100,
                                            chunksize=400,
                                            passes=100,  # 중복된 토픽이 나오는 경우, 에폭을 늘려야한다.(혹은 실제로 존재하는 토픽이 적을수도 있다.)
                                            alpha='auto',
                                            per_word_topics=True
                                            )


In [48]:
# Print the Keyword in the 10 topics
# ldamodel이 정한 토픽중 앞쪽 순서 10개의 토픽에 해당되는 키워드들입니다.
# 각 키워드들에는 가중치가 정해져있습니다.
# 이 가중치들을 바탕으로 문서의 토픽을 분류합니다.
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.003*"있는" + 0.002*"게임" + 0.002*"비가" + 0.002*"차량" + 0.002*"서울" + 0.002*"기자" '
  '+ 0.001*"자전거" + 0.001*"도로" + 0.001*"연구팀은" + 0.001*"경기"'),
 (1,
  '0.003*"있는" + 0.002*"지난해" + 0.002*"국내" + 0.002*"갤럭시s" + 0.002*"지난" + '
  '0.002*"스마트폰" + 0.002*"것으로" + 0.002*"위해" + 0.002*"기자" + 0.002*"올해"'),
 (2,
  '0.003*"기자" + 0.003*"했다" + 0.002*"지난" + 0.002*"밝혔다" + 0.002*"라고" + '
  '0.002*"경찰은" + 0.002*"그는" + 0.002*"있는" + 0.002*"배우" + 0.002*"a씨는"'),
 (3,
  '0.003*"있는" + 0.003*"많이" + 0.002*"한다" + 0.002*"있습니다" + 0.002*"서울" + '
  '0.002*"말했다" + 0.002*"하는" + 0.002*"많은" + 0.002*"가장" + 0.001*"같은"'),
 (4,
  '0.002*"있는" + 0.001*"위해" + 0.001*"같은" + 0.001*"중국은" + 0.001*"지구" + '
  '0.001*"기술" + 0.001*"연구팀은" + 0.001*"지난" + 0.001*"카메라" + 0.001*"밝혔다"'),
 (5,
  '0.003*"대회" + 0.003*"기자" + 0.003*"한국" + 0.002*"열린" + 0.002*"지난" + 0.002*"만에" '
  '+ 0.002*"이후" + 0.002*"올해" + 0.002*"경기" + 0.002*"번째"'),
 (6,
  '0.004*"지난" + 0.004*"있는" + 0.004*"대한" + 0.004*"말했다" + 0.003*"기자" + '
  '0.003*"밝혔다" + 0.003*"관련" + 0.003*"것으로

## TODO  1
위에서 확인해 볼 수 있듯이, 각 토픽은 전체 어휘의 확률분포로 나타낼 수 있습니다. 확률분포를 보면서, 어떤 토픽인지 가늠해볼 수 있나요? (아마도 **정치 경제, 사회, 생활/문화, 세계, 기술/IT, 연예, 스포츠** 중 하나일 것입니다!)

- 0번 토픽: 경제
- 1번 토픽:
- 2번 토픽:
- 3번 토픽:
- 4번 토픽:
- 5번 토픽:
- 6번 토픽:
- 7번 토픽:




## Visualise topic-keywords

좌측의 버블은 하나의 토픽을 의미합니다. 각 버블의 크기는 prevalent한 정도를 보여줍니다.

좋은 모델은 버블이 크고 서로 겹치지 않는 모습을 보여줍니다.

모델에 너무 많은 토픽을 넣게 되면 작은 버블들이 많이 겹치겠죠?

[그래프에 대한 자세한 설명](http://bl.ocks.org/AlessandraSozzi/raw/ce1ace56e4aed6f2d614ae2243aab5a5/)

[그래프를 개발한 논문](https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf)



In [49]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.183093  0.058288       1        1  20.724279
7     -0.157542  0.056975       2        1  17.944822
2     -0.044000 -0.224323       3        1  15.458434
3      0.075230 -0.088679       4        1  13.416297
1      0.052333  0.098364       5        1  10.635452
5      0.133765  0.070384       6        1   8.971024
0      0.072105  0.019237       7        1   6.575469
4      0.051202  0.009755       8        1   6.274224, topic_info=      Term        Freq       Total Category  logprob  loglift
1082   북한이  148.000000  148.000000  Default  30.0000  30.0000
201     그는  326.000000  326.000000  Default  29.0000  29.0000
12428   대회  104.000000  104.000000  Default  28.0000  28.0000
455     북한  154.000000  154.000000  Default  27.0000  27.0000
2059   경찰은  184.000000  184.000000  Default  26.0000  26.0000
...    ...         ...         ...      ...      ...      ...
4454    해외   14.972090   87.297134   Topic8  -7.2606   1.0056
1470    것이   16.522385  217.117188   Topic8  -7.1621   0.1930
42      대한   18.016407  597.078335   Topic8  -7.0755  -0.7320
205     기자   16.754514  779.291985   Topic8  -7.1481  -1.0710
218     대해   14.516202  396.883594   Topic8  -7.2915  -0.5397

[563 rows x 6 columns], token_table=       Topic      Freq Term
term                       
35652      3  0.986739   a씨
35653      3  0.976899  a씨가
14335      1  0.105187  a씨는
14335      3  0.894093  a씨는
36026      3  0.958089  a씨를
...      ...       ...  ...
356        2  0.488905   후보
356        8  0.236567   후보
357        1  0.073692  후보는
357        2  0.589535  후보는
357        8  0.313190  후보는

[1118 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 8, 3, 4, 2, 6, 1, 5])

### meaning of lambda

lambda의 의미: topic K에 속한 단어 W의 확률의 weight을 결정. (relative to its lift)
`
즉. 그래프에 표시되는 term의 확률 = (실제 단어 확률)*(1-람다) + (토픽별 확률)*(람다)
`

-> 1의 의미: 해당 토픽에서 익숙한 용어 랭킹을 토픽별 확률로 낮춤.

-> 0의 의미: 해당 단어의 확률로만 표현됨.

-> 최적의 값이 필요한 이유는 주제별 확률과 단어의 확률을 모두 반영해 토픽 이해에 최적화하기 위함.

-> 위 논문에서 0.6이라고 나오는데 이유는 

29명의 subjects에게 k와 람다를 바꿔가며 나온 결과를 평가했는데 거기서 나온 통계적 최적값이 약 0.67로 나왔다.
(Proportion of Correct Responses가 대략 70%)